# Create an Excel import job

You use an Excel import job to import data from a properly formatted Excel spreadsheet.

This example shows how to create an Excel import job request, submit it to the job
queue, and interact with the resulting Excel import job object returned by the server.

Information on how to create a properly formatted Excel import template is
outside the scope of this example. For information on using Excel to import data into
Granta MI, see the Granta MI documentation or consult your ACE representative.

## Connect to Granta MI

Import the ``Connection`` class and create the connection. For more information,
see the [Connect and access the job queue](0_Getting_started.ipynb) example.

In [1]:
from ansys.grantami.jobqueue import Connection

server_url = "http://my_grantami_server/mi_servicelayer"
client = Connection(server_url).with_credentials("user_name", "password").connect()

## Create an ``ExcelImportJobRequest`` object

The first step in importing an Excel file with the job queue is to create an
``ExcelImportJobRequest`` object. When creating this object, specify the name of the job and the
files to import. You can also specify an optional description and the scheduled execution
date, if the import should be deferred until that date and time.

Different job types require different input files. For example, an Excel import can use a
*template*, one or more *data* files, or *combined* files, which include both the template
and data files. You should specify any additional files to imported as file or picture attributes
as *attachment* files. You can provide these additional files as relative or absolute paths or as
``pathlib.Path`` objects.

In [2]:
from ansys.grantami.jobqueue import ExcelImportJobRequest

separate_excel_import_request = ExcelImportJobRequest(
    name="Excel Import (separate template and data files)",
    description="An example excel import job",
    template_file="assets/import_template.xlsx",
    data_files=["assets/data_file_1.xlsx", "assets/data_file_2.xlsx"],
)
separate_excel_import_request

<ExcelImportJobRequest: name: "Excel Import (separate template and data files)">

## Submit the job to the server
Next, submit the jobs to the server. There are two methods for submitting job
requests:

* ``create_job()``: Submit the job request to the server and immediately return an
  ``AsyncJob`` object in the *pending* state.
* ``create_job_and_wait()``: Submit the job request to the server and block until the job
   either completes or fails. Return an ``AsyncJob`` object in the *succeeded* or *failed* state.

This example uses the ``create_job_and_wait()`` method. For an example that shows
how to create and submit a job that runs asynchronously, see
[Schedule and modify jobs](4_Scheduling_and_modifying_jobs.ipynb).

In [3]:
completed_job = client.create_job_and_wait(separate_excel_import_request)

## Access output files
Finally, access the results of the job. Import jobs typically create log files, but the exact type
of files generated varies based on the type of import template. In this case, the files are all
plain text.

Access the list of files generated by the job with the ``output_file_names`` property. This
returns a list of file names.

In [4]:
completed_job.output_file_names

['Excel Import (separate template and data files).log']

In general, an Excel import job includes a log of the import operation on the server as ``<job name>.log``.

This cell shows how to access the content of the log file as ``bytes`` using the
``AsyncJob.get_file_content()`` method:

In [5]:
log_file_name = next(name for name in completed_job.output_file_names if "log" in name)
log_file_content = completed_job.get_file_content(log_file_name)
log_file_string = log_file_content.decode("utf-8")
print(f"{log_file_name} (first 200 characters):")
print(f"{log_file_string[:500]}...")

Excel Import (separate template and data files).log (first 200 characters):
﻿2026-02-11 16:00:24,370 [20] INFO  Task started: Template:'import_template', Step:'Pre-Import Validation: Problematic Values & Functions', Datafile:'data_file_1', Culture:'en-US'
2026-02-11 16:00:24,425 [20] INFO  Task finished.
2026-02-11 16:00:24,440 [20] INFO  Task started: Template:'import_template', Step:'Pre-Import Validation: Problematic Values & Functions', Datafile:'data_file_2', Culture:'en-US'
2026-02-11 16:00:24,446 [20] INFO  Task finished.
2026-02-11 16:00:24,453 [20] INFO  Ta...
